# The Battle Of Neighbourhoods

## 1 Business Problem Section

### 1.1 Background

New York city is world's major financial and cultural centre. As a result of this, real estate properties see lot of activity in terms of buying and selling. 
Due to financial cricis of 2008, NYC has seen a bit of downturn in real estate prices but recently it has seen some recovery. Investors looking for real estate investment are very curious to know the best way to invest in the current scenario. 

### 1.2 Business Problem :

Homebuyer client want optimal recommendation based on his requirements to buy a house in NYC. The house should meet his requirements, that is to be able to take a decision 
of buying in NYC.

## 2. Data Description and Methodology

### 2.1 Data Description:

Using Data Science techniques learnt in this course and using FourSquare location data we will provide recommendations to client.
We are going to cluster New York neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment.
Also we will recommend profitable venues venues i.e. pharmacy, restaurants, hospitals & grocery stores.

The Department of Finance (DOF) maintains records for all property sales in New York City, including sales of family homes in each borough(https://data.cityofnewyork.us/api/views/948r-3ads/rows.csv?accessType=DOWNLOAD).
This list includes all sales of 1-, 2-, and 3-Family Homes' from January 1st, 2009 to December 31, 2009, whose sale price is equal to or more than $150,000. The Building Class Category for Sales is based on the Building Class at the time of the sale.
To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on New York properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

### 2.2 Methodology

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

In [3]:
# Importing Libraries
#Beautifulsoup library helps in web scraping data from webpage
from bs4 import BeautifulSoup
#lxml library is the parser used to parse the content from diffrent HTML Tags
import lxml
# Requests library helps in getting the content of the webpage
import requests as req
# library to handle data in a vectorized manner
import numpy as np
#library for Data Analysis
import pandas as pd
# library to handle JSON files
import json 
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
# library to handle requests
import requests 
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib as plt
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
# map rendering library
import folium 
# library to find median of List
from numpy import median
print('Libraries imported.')

Libraries imported.


In [4]:
# Download the Neighbourhood of NewYork with price dataset
!wget -O ny_neighbourhood.csv  https://data.cityofnewyork.us/api/views/948r-3ads/rows.csv?accessType=DOWNLOAD

--2019-08-04 07:54:26--  https://data.cityofnewyork.us/api/views/948r-3ads/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘ny_neighbourhood.csv’

    [ <=>                                   ] 18,612      --.-K/s   in 0s      

2019-08-04 07:54:27 (194 MB/s) - ‘ny_neighbourhood.csv’ saved [18612]

